## I. Import thư viện

In [2]:
from bs4 import BeautifulSoup
import time
import random
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException
from tqdm import tqdm
import re
import pandas as pd
from unicodedata import normalize
import os

## II. Lấy các URL từ Chrome

### 1. Crawl link khách sạn theo từng tỉnh thành
rồi lưu vào file txt tương ứng

In [ ]:

def scroll_and_wait_for_button(driver, max_attempts=3):
    attempts = 0
    while attempts < max_attempts:
        # Cuộn xuống cuối trang
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        
        # Chờ ngẫu nhiên 2-4 giây để nút có thể xuất hiện
        wait_time = random.uniform(4.0, 8.0)
        time.sleep(wait_time)
        
        try:
            # Thử tìm nút "Tải thêm kết quả"
            load_more_button = WebDriverWait(driver, 5).until(
                EC.presence_of_element_located((By.XPATH, "//button[contains(., 'Tải thêm kết quả')]")))
            
            # Cuộn đến nút và click
            driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", load_more_button)
            time.sleep(1)  # Đợi ổn định
            load_more_button.click()
            print(f"Đã click vào nút 'Tải thêm kết quả' (lần {attempts + 1})")
            
            # Chờ nội dung mới tải xong
            time.sleep(random.uniform(3.0, 5.0))
            attempts = 0  # Reset counter sau khi click thành công
            
        except (TimeoutException, NoSuchElementException):
            print(f"Không tìm thấy nút 'Tải thêm kết quả' (lần thử {attempts + 1}/{max_attempts})")
            attempts += 1
            
            # Nếu đã thử nhiều lần mà không thấy nút thì dừng
            if attempts >= max_attempts:
                print("Đã thử tối đa số lần, dừng tìm nút.")
                break

def get_hotel_links(driver):
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    hotel_links = []
    
    # Tìm các thẻ div chứa thông tin khách sạn
    hotel_cards = soup.find_all('div', {'data-testid': 'property-card'})
    
    for card in hotel_cards:
        # Tìm thẻ a chứa link khách sạn
        link_tag = card.find('a', {'data-testid': 'property-card-desktop-single-image'})
        if link_tag and 'href' in link_tag.attrs:
            href = link_tag['href']
            clean_url = href.split('?')[0]
            if clean_url not in hotel_links:
                hotel_links.append(clean_url)
    
    return hotel_links

def main():
    countries = ['danang'] #  đưa các tỉnh cần crawl vào list countries
    
#     "hanoi", "haiphong", "danang", "hochiminh", "cantho", "angiang", "bacgiang", "backan",
#     "baclieu", "bacninh", "baria-vungtau", "bentre", "binhdinh", "binhduong", "binhphuoc", "binhthuan",
#     "camau", "caobang", "daklak", "daknong", "dienbien", "dongnai", "dongthap", "gialai", "hagiang",
#     "hanam", "hatinh", "haiduong", "haugiang", "hoabinh", "hungyen", "khanhhoa", "kiengiang",
#     "kontum", "laichau", "lamdong", "langson", "laocai", "longan", "namdinh", "nghean", "ninhbinh",
#     "ninhthuan", "phutho", "phuyen", "quangbinh", "quangnam", "quangngai", "quangninh", "quangtri",
#     "soctrang", "sonla", "tayninh", "thaibinh", "thainguyen", "thanhhoa", "hue",
#     "tiengiang", "travinh", "tuyenquang", "vinhlong", "vinhphuc", "yenbai"
    
    options = Options()
    options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36")
    options.add_argument("--start-maximized")
    
    for country in countries:
        all_hotel_links = []
        driver = webdriver.Chrome(options=options)
        try:
            search_url = f'https://www.booking.com/searchresults.vi.html?ss={country}'
            driver.get(search_url)
            
            # Đợi trang tải ban đầu
            time.sleep(random.uniform(3, 5))
            
            # Xử lý nút "Tải thêm kết quả"
            scroll_and_wait_for_button(driver, max_attempts=8)
            
            country_hotel_links = get_hotel_links(driver)
            new_links = [url for url in country_hotel_links if url not in all_hotel_links]
            all_hotel_links.extend(new_links)
            
            print(f"Tìm thấy {len(country_hotel_links)} khách sạn tại {country}.")

            # Xuất file txt mỗi tỉnh
            with open(f"hotel_links_booking_{country}.txt", "w", encoding="utf-8") as f:
                for url in all_hotel_links:
                    f.write(url + "\n")
            print(f"Tổng cộng tìm thấy {len(all_hotel_links)} khách sạn.")
        finally:
            driver.quit()

if __name__ == "__main__":
    main()

### 2. Hàm crawl các thuộc tính khách sạn

In [3]:
def crawl_hotels_vn(url, driver, is_first=False):
    # options = Options()options=options
    # options.add_argument("--headless")  # Chạy ẩn trình duyệtrandom.uniform(3,7)
    
    driver.get(url)
    if is_first:
        print("Lần đầu tiên — đợi để bạn đăng nhập...")
        time.sleep(60)
    else:
        time.sleep(random.uniform(3, 7))
    soup_hotel_vn = BeautifulSoup(driver.page_source, 'html.parser')
    #driver.quit()

# url khách sạn
    hotel_url = url
    
# tên khách sạn
    hotel_name = soup_hotel_vn.find('h2', class_='ddb12f4f86 pp-header__title').text

# giá chung
    price_span = soup_hotel_vn.find_all('span', class_='prco-valign-middle-helper')
    prices = []
    for span in price_span:
        text = span.get_text(strip=True)
        number_str = re.sub(r'\D', '', text)
        prices.append(int(number_str))
    overview_price = None
    if prices:
        avg_price = sum(prices) / len(prices)
        # định dạng tiền tệ VN
        overview_price = "{:,.0f} VND".format(avg_price).replace(",", ".")

# địa chỉ
    address_wrapper = soup_hotel_vn.find('div', class_='b99b6ef58f cb4b7a25d9')
    if address_wrapper:
        address = address_wrapper.find(string=True, recursive=False).strip()

# điểm đánh giá chung
    overall_rating_wrapper = soup_hotel_vn.find('div', {'data-testid': 'PropertyReviewsRegionBlock'})
    overall_rating = None
    if overall_rating_wrapper:
        overall_rating = overall_rating_wrapper.find('div', class_='f63b14ab7a dff2e52086')
    if overall_rating:
        overall_rating = overall_rating.text

# điểm đánh giá 7 khía cạnh (nhân viên phục vụ, tiện nghi, sạch sẽ, thoải mái, đáng giá tiền, địa điểm, wifi miễn phí)
    review_div = soup_hotel_vn.find('div', {'data-testid': 'ReviewSubscoresDesktop'})
    staff = facilities = cleanliness = comfort = value_for_money = location = free_wifi = subscore = None
    
    if review_div:
        next_div = review_div.find_next_sibling('div')

        if next_div:
            subscore_label = next_div.find_all('span', class_='d96a4619c0')
            subscore = next_div.find_all('div', class_='a9918d47bf f87e152973')

    if subscore:
        for label, value in zip(subscore_label, subscore):
            label = normalize('NFC',(label.get_text(strip=True))).strip()
            value = value.get_text(strip=True)
            if label == "Nhân viên phục vụ":
                staff = value
            elif label == "Tiện nghi":
                facilities = value
            elif label == "Sạch sẽ":
                cleanliness = value
            elif label == "Thoải mái":
                comfort = value
            elif label == "Đáng giá tiền":
                value_for_money = value
            elif label == "Địa điểm":
                location = value
            elif label == "WiFi miễn phí":
                free_wifi = value

# tiện nghi ưa chuộng nhất
    facilities_wrapper = soup_hotel_vn.find('div', attrs={'data-testid': 'property-most-popular-facilities-wrapper'})
    popular_facilities = None
    
    if facilities_wrapper:
        facility_items = facilities_wrapper.find_all('li')
        facility_texts = [li.get_text(strip=True) for li in facility_items]
        popular_facilities = ', '.join(facility_texts)

# thời gian nhận phòng & thời gian trả phòng
    left_column = soup_hotel_vn.find_all('div', class_='a776c0ae8b')
    right_column = soup_hotel_vn.find_all('div', class_='c92998be48')
    checkin_time = checkout_time = None
    for left, right in zip(left_column, right_column):
        value = right.find('div', class_='b99b6ef58f')
        label = left.get_text(strip=True)
        if label == "Nhận phòng":
            checkin_time = value.get_text(strip=True)
        elif label == "Trả phòng":
            checkout_time = value.get_text(strip=True)

    return [hotel_url, hotel_name, overview_price, address, overall_rating, staff, facilities, cleanliness, comfort, value_for_money, location, free_wifi, popular_facilities, checkin_time, checkout_time]


### 3. Crawl thuộc tính các khách sạn theo tỉnh
* Đọc từng file txt (từng tỉnh) để crawl lần lượt
* Xuất file csv thuộc tính khách sạn của mỗi tỉnh

In [ ]:
# execute hàm craw_hotels_vn (cell trên) truớc khi chạy cell này

def crawl_hotels_for_provinces(provinces):
    os.makedirs('hotels_data', exist_ok=True)

    # tạo 2 folder hotels_data và hotel_links_booking trong cùng thư mục với file này

    input_folder = 'hotel_links_booking'  # chứa các file hotel_links_booking_<province>.txt trên drive
    output_folder = 'hotels_data' # chứa các file <province>_hotels.csv sau khi crawl
    
    driver = webdriver.Chrome()
    
    try:
        for province in provinces:
            input_path = os.path.join(input_folder, f'hotel_links_booking_{province.lower()}.txt')
            output_path = os.path.join(output_folder, f'{province.lower()}_hotels.csv')
            
            if not os.path.exists(input_path):
                print(f"Không tìm thấy file {input_path}")
                continue
            
            # Đọc tất cả url trong file input
            with open(input_path, 'r', encoding='utf-8') as f:
                hotel_links = [line.strip() for line in f if line.strip()]
            
            hotel_url, hotel_name, overview_price, address, overall_rating, staff, facilities, cleanliness  = [], [], [], [], [], [], [], []
            comfort, value_for_money, location, free_wifi, popular_facilities, checkin_time, checkout_time = [], [], [], [], [], [], []
            
            for idx, url in enumerate(tqdm(hotel_links)):
                is_first = (idx==0)
                try:
                    hotels_vn_data = crawl_hotels_vn(url, driver, is_first)
                    hotel_url.append(url)
                    hotel_name.append(hotels_vn_data[1])
                    overview_price.append(hotels_vn_data[2])
                    address.append(hotels_vn_data[3])
                    overall_rating.append(hotels_vn_data[4])
                    staff.append(hotels_vn_data[5])
                    facilities.append(hotels_vn_data[6])
                    cleanliness.append(hotels_vn_data[7])
                    comfort.append(hotels_vn_data[8])
                    value_for_money.append(hotels_vn_data[9])
                    location.append(hotels_vn_data[10])
                    free_wifi.append(hotels_vn_data[11])
                    popular_facilities.append(hotels_vn_data[12])
                    checkin_time.append(hotels_vn_data[13])
                    checkout_time.append(hotels_vn_data[14])
                except Exception as e:
                    print(f"Lỗi ở {url}: {e}")
            
            # Tạo DataFrame
            df = pd.DataFrame({
                'Hotel URL': hotel_url,
                'Hotel Name': hotel_name,
                'Overview Price': overview_price,
                'Address': address,
                'Overall Rating': overall_rating,
                'Staff': staff,
                'Facilities': facilities,
                'Cleanliness': cleanliness,
                'Comfort': comfort,
                'Value for Money': value_for_money,
                'Location': location,
                'Free Wifi': free_wifi,
                'Popular Facilities': popular_facilities,
                'Checkin Time': checkin_time,
                'Checkout Time': checkout_time
            })
            
            # Lưu DataFrame ra file csv
            df.to_csv(output_path, index=False, encoding='utf-8-sig')
            print(f"Hoàn thành crawl {province}, lưu dữ liệu tại {output_path}")
    finally:
        # driver.quit()
        print("Đã xong.")

In [ ]:
#     "hanoi", "haiphong", "danang", "hochiminh", "cantho", "angiang", "bacgiang", "backan",
#     "baclieu", "bacninh", "baria-vungtau", "bentre", "binhdinh", "binhduong", "binhphuoc", "binhthuan",
#     "camau", "caobang", "daklak", "daknong", "dienbien", "dongnai", "dongthap", "gialai", "hagiang",
#     "hanam", "hatinh", "haiduong", "haugiang", "hoabinh", "hungyen", "khanhhoa", "kiengiang",
#     "kontum", "laichau", "lamdong", "langson", "laocai", "longan", "namdinh", "nghean", "ninhbinh",
#     "ninhthuan", "phutho", "phuyen", "quangbinh", "quangnam", "quangngai", "quangninh", "quangtri",
#     "soctrang", "sonla", "tayninh", "thaibinh", "thainguyen", "thanhhoa", "hue",
#     "tiengiang", "travinh", "tuyenquang", "vinhlong", "vinhphuc", "yenbai"

# Gọi hàm đưa vào list tỉnh muốn crawl
crawl_hotels_for_provinces(["khanhhoa", "kiengiang",
     "kontum", "laichau", "lamdong", "langson", "laocai", "longan", "namdinh", "nghean", "ninhbinh",
     "ninhthuan", "phutho", "phuyen", "quangbinh", "quangnam", "quangngai", "quangninh", "quangtri",
     "soctrang", "sonla", "tayninh", "thaibinh", "thainguyen", "thanhhoa",
     "tiengiang", "travinh", "tuyenquang", "vinhlong", "vinhphuc", "yenbai"])